# Training a new tokenizer from an old one

Install the Transformers, Datasets, and Evaluate libraries to run this notebook.

In [ ]:
!pip install datasets evaluate transformers[sentencepiece]
!apt install git-lfs

You will need to setup git, adapt your email and name in the following cell.

In [ ]:
# !git config --global user.email "you@example.com"
# !git config --global user.name "Your Name"

You will also need to be logged in to the Hugging Face Hub. Execute the following and enter your credentials.

In [1]:
from huggingface_hub import notebook_login
notebook_login()

<small>Если языковая модель недоступна для языка, который вас интересует, или если ваш корпус сильно отличается от того, на котором была обучена ваша языковая модель, вам, скорее всего, захочется переобучить модель с нуля, используя токенизатор, адаптированный к вашим данным. Это потребует обучения нового токенизатора на вашем наборе данных. Но что именно это означает? Когда мы впервые рассмотрели токенизаторы в Главе 2, мы увидели, что большинство моделей Transformer используют алгоритм токенизации подслов. Чтобы определить, какие подслова представляют интерес и встречаются чаще всего в рассматриваемом корпусе, токенизатор должен внимательно изучить все тексты в корпусе — процесс, который мы называем обучением. Точные правила, которые управляют этим обучением, зависят от типа используемого токенизатора, и мы рассмотрим три основных алгоритма далее в этой главе.</small>

<small>Обучение токенизатора — это не то же самое, что обучение модели! Обучение модели использует стохастический градиентный спуск, чтобы сделать потерю немного меньше для каждой партии. Оно рандомизировано по своей природе (это означает, что вам нужно установить некоторые начальные значения, чтобы получить те же результаты при выполнении одного и того же обучения дважды). Обучение токенизатора — это статистический процесс, который пытается определить, какие подслова лучше всего выбрать для данного корпуса, и точные правила, используемые для их выбора, зависят от алгоритма токенизации. Оно детерминировано, то есть вы всегда получаете те же результаты при обучении с тем же алгоритмом на том же корпусе.</small>

## Сборка корпуса

<small>В 🤗 Transformers есть очень простой API, который можно использовать для обучения нового токенизатора с теми же характеристиками, что и у существующего: AutoTokenizer.train_new_from_iterator(). Чтобы увидеть это в действии, предположим, что мы хотим обучить GPT-2 с нуля, но на языке, отличном от английского. Нашей первой задачей будет собрать много данных на этом языке в обучающем корпусе. Чтобы предоставить примеры, которые все смогут понять, мы не будем использовать здесь язык, такой как русский или китайский, а вместо этого будем использовать специализированный английский язык: код Python.

Библиотека 🤗 Datasets может помочь нам собрать корпус исходного кода Python. Мы будем использовать обычную функцию load_dataset() для загрузки и кэширования набора данных CodeSearchNet. Этот набор данных был создан для задачи CodeSearchNet и содержит миллионы функций из библиотек с открытым исходным кодом на GitHub на нескольких языках программирования. Здесь мы загрузим часть Python этого набора данных:</small>

In [5]:
from datasets import load_dataset

# This can take a few minutes to load, so grab a coffee or tea while you wait!
raw_datasets = load_dataset("code_search_net", "python")

The repository for code_search_net contains custom code which must be executed to correctly load the dataset. You can inspect the repository content at https://hf.co/datasets/code_search_net.
You can avoid this prompt in future by passing the argument `trust_remote_code=True`.

Do you wish to run the custom code? [y/N] y


python.zip:   0%|          | 0.00/941M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/412178 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/22176 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/23107 [00:00<?, ? examples/s]

In [6]:
raw_datasets["train"]

Dataset({
    features: ['repository_name', 'func_path_in_repository', 'func_name', 'whole_func_string', 'language', 'func_code_string', 'func_code_tokens', 'func_documentation_string', 'func_documentation_tokens', 'split_name', 'func_code_url'],
    num_rows: 412178
})

<small>Мы видим, что набор данных разделяет docstrings от кода и предлагает токенизацию обоих. Здесь мы просто используем столбец whole_func_string для обучения нашего токенизатора. Мы можем посмотреть на пример одной из этих функций, проиндексировав разделение train:</small>

In [7]:
print(raw_datasets["train"][123456]["whole_func_string"])

def last_rate_limit(self):
        """
        A `dict` of the rate limit information returned in the most recent
        response, or `None` if no requests have been made yet.  The `dict`
        consists of all headers whose names begin with ``"RateLimit"`` (case
        insensitive).

        The DigitalOcean API specifies the following rate limit headers:

        :var string RateLimit-Limit: the number of requests that can be made
            per hour
        :var string RateLimit-Remaining: the number of requests remaining until
            the limit is reached
        :var string RateLimit-Reset: the Unix timestamp for the time when the
            oldest request will expire from rate limit consideration
        """
        if self.last_response is None:
            return None
        else:
            return {k:v for k,v in iteritems(self.last_response.headers)
                        if k.lower().startswith('ratelimit')}


<small>Первое, что нам нужно сделать, это преобразовать набор данных в итератор списков текстов — например, список списка текстов. Использование списков текстов позволит нашему токенизатору работать быстрее (обучение на пакетах текстов вместо обработки отдельных текстов по одному), и это должен быть итератор, если мы хотим избежать одновременного хранения всего в памяти. Если ваш корпус огромен, вам нужно будет воспользоваться тем фактом, что 🤗 Datasets не загружает все в оперативную память, а сохраняет элементы набора данных на диске.

Выполнение следующих действий создаст список списков по 1000 текстов в каждом, но загрузит все в память:</small>

In [ ]:
# Don't uncomment the following line unless your dataset is small!
# Не раскомментируйте следующую строку, если ваш набор данных не маленький!

# training_corpus = [raw_datasets["train"][i: i + 1000]["whole_func_string"] for i in range(0, len(raw_datasets["train"]), 1000)]

In [ ]:
# Не запускаем!!!!!!!!!!!
training_corpus = (
    raw_datasets["train"][i : i + 1000]["whole_func_string"]
    for i in range(0, len(raw_datasets["train"]), 1000)
)

Эта строка кода не извлекает никаких элементов из набора данных; она просто создает объект, который вы можете использовать в цикле for Python. Тексты будут загружены только тогда, когда они вам понадобятся (то есть, когда вы находитесь на шаге цикла for, который их требует), и только 1000 текстов за раз будут загружены. Таким образом, вы не исчерпаете всю свою память, даже если обрабатываете огромный набор данных.

Проблема с объектом-генератором заключается в том, что его можно использовать только один раз. Поэтому вместо того, чтобы дважды выдавать нам список первых 10 цифр:

In [ ]:
gen = (i for i in range(10))
print(list(gen))
print(list(gen))

[0, 1, 2, 3, 4, 5, 6, 7, 8, 9]
[]

мы получаем их один раз, а затем пустой список:

Вот почему вместо этого мы определяем функцию, которая возвращает генератор (`Это не сам генератор, а функция, создающая генератор`).  

При вызове `get_training_corpus()` создается генератор-выражение (generator expression), который итерируется по raw_datasets["train"] с шагом 1000.
⏳ Важно: Этот генератор ленивый `(lazy) – он не выполняется сразу, а только при итерации`.

 (мы собираем порциями по 1000 записей за раз только часть данных из "сырых" данных (raw_datasets), именно - данные одного столбца: `raw_datasets["train"]["whole_func_string"]`)

In [ ]:
def get_training_corpus():
    return (
        raw_datasets["train"][i : i + 1000]["whole_func_string"] for i in range(0, len(raw_datasets["train"]), 1000)
    )

Как альтернатвный вариант, можно определить функцию генератора внутри цикла for, используя оператор `yield`:

In [15]:
def get_training_corpus():
    dataset = raw_datasets["train"]
    for start_idx in range(0, len(dataset), 1000):
        samples = dataset[start_idx : start_idx + 1000]
        yield samples["whole_func_string"]

Создание объекта генератора

In [16]:
training_corpus = get_training_corpus()

Здесь вызывается функция `get_training_corpus()`, которая возвращает генератор-выражение.

__На этом этапе ничего не происходит__ – данные еще не загружены и не обработаны.
Генератор готов к работе, но __он "ждет", пока кто-то начнет его запрашивать__.

## Обучение нового токенизатора

Теперь, когда у нас есть корпус в форме итератора партий текстов, мы готовы обучить новый токенизатор. Для этого нам сначала нужно загрузить токенизатор, который мы хотим связать с нашей моделью (здесь GPT-2):

In [17]:
from transformers import AutoTokenizer

old_tokenizer = AutoTokenizer.from_pretrained("gpt2")

The cache for model files in Transformers v4.22.0 has been updated. Migrating your old cache. This is a one-time only operation. You can interrupt this and resume the migration later on by calling `transformers.utils.move_cache()`.


0it [00:00, ?it/s]

tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/665 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

Несмотря на то, что мы собираемся обучить новый токенизатор, это хорошая идея, чтобы не начинать все с нуля. Таким образом, нам не придется ничего указывать об алгоритме токенизации или специальных токенах, которые мы хотим использовать; наш новый токенизатор будет точно таким же, как GPT-2, и единственное, что изменится, — это словарь, который будет определен обучением на нашем корпусе.

Сначала давайте посмотрим, как старый токенизатор будет обрабатывать пример функции:

In [ ]:
example = '''def add_numbers(a, b):
    """Add the two numbers `a` and `b`."""
    return a + b'''

tokens = old_tokenizer.tokenize(example)
tokens

Этот токенизатор имеет несколько специальных символов, таких как Ġ и Ċ, которые обозначают пробелы и переводы строк соответственно. Как мы видим, это не слишком эффективно: токенизатор возвращает отдельные токены для каждого пробела, когда он мог бы группировать уровни отступов (поскольку наборы из четырех или восьми пробелов будут очень распространены в коде). Он также немного странно разделил имя функции, не привыкнув видеть слова с символом _.

Использование генератора в train_new_from_iterator

Давайте обучим новый токенизатор и посмотрим, решит ли он эти проблемы. Для этого мы воспользуемся методом

In [19]:
tokenizer = old_tokenizer.train_new_from_iterator(training_corpus, 52000)

Что происходит?
Функция `train_new_from_iterator` начинает итерировать по training_corpus.  
__Это запускает генератор.__  
`training_corpus` начинает лениво (по 1000 элементов за раз) извлекать `"whole_func_string"` из `raw_datasets["train"]`.  

__Первый раз__ запрашивается `next(training_corpus)`, генератор выполняет:  
`raw_datasets["train"][0:1000]["whole_func_string"]`  
Возвращается первая порция из 1000 элементов.  
train_new_from_iterator их обрабатывает.  

Генератор продолжает работу  
__При следующем__ `next(training_corpus)`, выполняется:
`raw_datasets["train"][1000:2000]["whole_func_string"]`  
И так далее, пока не дойдет до конца.
Когда элементы заканчиваются, генератор выбрасывает `StopIteration`, и `train_new_from_iterator` завершает работу.

<small>Эта команда может занять некоторое время, если ваш корпус очень большой, но для этого набора данных из 1,6 ГБ текстов она молниеносно быстрая (1 минута 16 секунд на процессоре AMD Ryzen 9 3900X с 12 ядрами).

Обратите внимание, что AutoTokenizer.train_new_from_iterator() работает только в том случае, если используемый вами токенизатор является «быстрым» токенизатором. Как вы увидите в следующем разделе, библиотека 🤗 Transformers содержит два типа токенизаторов: некоторые написаны исключительно на Python, а другие (быстрые) поддерживаются библиотекой 🤗 Tokenizers, которая написана на языке программирования Rust. Python — это язык, который чаще всего используется для приложений науки о данных и глубокого обучения, но когда что-то нужно распараллелить, чтобы оно было быстрым, это должно быть написано на другом языке. Например, умножение матриц, которое лежит в основе вычисления модели, написано на CUDA, оптимизированной библиотеке C для графических процессоров.  
Обучение совершенно нового токенизатора на чистом Python было бы мучительно медленным, поэтому мы разработали библиотеку 🤗 Tokenizers. Обратите внимание, что так же, как вам не нужно изучать язык CUDA, чтобы иметь возможность выполнять свою модель на пакете входных данных на GPU, вам не нужно изучать Rust, чтобы использовать быстрый токенизатор. Библиотека 🤗 Tokenizers предоставляет привязки Python для многих методов, которые внутренне вызывают некоторый фрагмент кода в Rust; например, для распараллеливания обучения вашего нового токенизатора или, как мы видели в Главе 3, токенизации пакета входных данных.  
Большинство моделей Transformer имеют доступный быстрый токенизатор (есть некоторые исключения, которые вы можете проверить здесь), и API AutoTokenizer всегда выбирает для вас быстрый токенизатор, если он доступен. В следующем разделе мы рассмотрим некоторые другие специальные функции быстрых токенизаторов, которые будут действительно полезны для таких задач, как классификация токенов и ответы на вопросы. Однако, прежде чем углубляться в это, давайте попробуем наш совершенно новый токенизатор на предыдущем примере:</small>

In [ ]:
tokens = tokenizer.tokenize(example)
tokens

Здесь мы снова видим специальные символы Ġ и Ċ, которые обозначают пробелы и переводы строк, но мы также можем видеть, что наш токенизатор узнал некоторые токены, которые весьма специфичны для корпуса функций Python: например, есть токен ĊĠĠĠ, который представляет отступ, и токен Ġ""", который представляет три кавычки, которые начинают строку документации. Токенизатор также правильно разделил имя функции на _. Это довольно компактное представление; сравнительно, использование токенизатора на простом английском языке в том же примере даст нам более длинное предложение:

In [21]:
print(len(tokens))
print(len(old_tokenizer.tokenize(example)))

27
36


In [ ]:
example_2 = """class LinearLayer():
    def __init__(self, input_size, output_size):
        self.weight = torch.randn(input_size, output_size)
        self.bias = torch.zeros(output_size)

    def __call__(self, x):
        return x @ self.weights + self.bias
    """
tokenizer.tokenize(example_2)

В дополнение к токену, соответствующему отступу, здесь мы также видим токен для двойного отступа: ĊĠĠĠĠĠĠĠĠ. Специальные слова Python, такие как class, init, call, self и return, каждое токенизировано как один токен, и мы можем видеть это, а также разделение на _ и . токенизатор правильно разделяет даже имена в верблюжьем стиле: LinearLayer токенизирован как ["ĠLinear", "Layer"].

## Сохранение токенизатора

Чтобы использовать его позже, нам нужно сохранить наш новый токенизатор. Как и для моделей, это делается с помощью метода `save_pretrained():`

In [23]:
tokenizer.save_pretrained("code-search-net-tokenizer")

('code-search-net-tokenizer/tokenizer_config.json',
 'code-search-net-tokenizer/special_tokens_map.json',
 'code-search-net-tokenizer/vocab.json',
 'code-search-net-tokenizer/merges.txt',
 'code-search-net-tokenizer/added_tokens.json',
 'code-search-net-tokenizer/tokenizer.json')

Это создаст новую папку с именем code-search-net-tokenizer, которая будет содержать все файлы, которые необходимо перезагрузить токенизатору.  

Если вы хотите поделиться этим токенизатором с коллегами и друзьями, вы можете загрузить его в Hub, войдя в свою учетную запись. Если вы работаете в блокноте, есть удобная функция, которая поможет вам в этом:

In [ ]:
from huggingface_hub import notebook_login

notebook_login()

Это отобразит виджет, в котором вы можете ввести свои учетные данные для входа в Hugging Face. Если вы не работаете в блокноте, просто введите следующую строку в терминале:

In [ ]:
huggingface-cli login

После входа в систему вы можете отправить свой токенизатор, выполнив следующую команду:

In [ ]:
tokenizer.push_to_hub("code-search-net-tokenizer")

Это создаст новый репозиторий в вашем пространстве имен с именем code-search-net-tokenizer, содержащий файл токенизатора. Затем вы можете загрузить токенизатор откуда угодно с помощью метода from_pretrained():

In [ ]:
# Replace "huggingface-course" below with your actual namespace to use your own tokenizer
tokenizer = AutoTokenizer.from_pretrained("huggingface-course/code-search-net-tokenizer")

В дпльнейшем более подробно рассмотрим быстрые токенизаторы и подробно изучим, что на самом деле происходит, когда мы вызываем метод train_new_from_iterator().